# Network Analysis: caculating accessible area using Euclidean distance and Manhattan distance

This Jupyter notebook investigate the differences between Euclidean distance and Manhattan distance and between Buffer and Convex Hull. To exemplify, we will examine the census block group that is accessible to a stadium (Ralph Engelstad Arena) in Grand Forks County, North Dakota.

### Data: 
* Census Block: https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.html <br>
* Open Street Map: https://www.openstreetmap.org/

### Steps:
1. Calculate Euclidean Distance
2. Calculate accessible census blocks using Euclidean distance
3. Calculate Manhattan Distance
4. Calculate accessible census blocks using Manhattan Distance


# Import Packages
A Python package is a way of organizing related Python modules into a single directory hierarchy. It provides a mechanism for grouping Python code files, resources, and configuration settings in a structured manner, making it easier to manage and distribute code. They also facilitate code reuse and distribution by allowing developers to bundle related functionality together and share it with others.

The following packages are used in this notebook:<br>
`networkx` is a Python package for the creation, manipulation, and study of the structure, dynamics, and functions of complex networks (e.g., road networks). <br>
source: https://networkx.org/documentation/stable/index.html<br>

`osmnx` is a Python package to retrieve, model, analyze, and visualize street networks from OpenStreetMap. <br>
source: https://osmnx.readthedocs.io/<br>

`geopy` is a Python package to locate the coordinates of addresses, cities, countries, and landmarks across the globe using third-party geocoders and other data sources. <br>
source: https://geopy.readthedocs.io/<br>

`math` is a built-in module that provides access to the mathematical functions defined by the C standard. <br>
soure: https://docs.python.org/3/library/math.html<br>

In [ ]:
# New Packages
import networkx as nx
import osmnx as ox
import geopy
import math

# Packages that investigated in the previous project
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt

# Etc
import warnings
warnings.filterwarnings('ignore')

# 1. Calculate Euclidean Distance

## 1.1. Geocoding two addresses
With the help of the geopy package, we can find the latitude and longitude of the stadium in Grand Forks County, North Dakota. 

Syntax: 
```python
geolocator = geopy.geocoders.Nominatim(user_agent="my_first_geocoder")
result = geolocator.geocode("TYPE AN ADDRESS")
```

Note: this example uses a geocoding engine provided by OpenStreetMap but not as good as a commercial geocoding engine. If the address is searchable on OpenStreetMap, you can extract the latitude and longitude of the address. If the address is not searchable, you can use a commercial geocoding engine such as Google Maps, Bing Maps, or MapQuest.


In [ ]:
# Initate a geocoding engine
geolocator = geopy.geocoders.Nominatim(user_agent="my_first_geocoder")

stadium = geolocator.geocode("775 Hamline St, Grand Forks, ND 58203")
stadium

In [ ]:
# Geocoded result is stored in the latitude and longitude attributes
stadium.latitude, stadium.longitude

In [ ]:
# Geocoding a different location (a drom at University of North Dakota)
dorm = geolocator.geocode("3601 University Ave, Grand Forks, ND 58202")
print(dorm.latitude, dorm.longitude)

## 1.2. Save geocoded results into GeoDataFrame

In [ ]:
# You can save the geocoded result in a dictionary to be converted into a (Geo)DataFrame
location_dict = {
    'Name': ['Dorm', 'Stadium'],
    'Latitude': [dorm.latitude, stadium.latitude],
    'Longitude': [dorm.longitude, stadium.longitude]
}
location_dict

Syntax to define a GeoDataFrame
```python
gdf = gpd.GeoDataFrame(DICTIONARY, 
                      geometry=gpd.points_from_xy(DICTIONARY['LONGITUDE COLUMN'], DICTIONARY.['LATITUDE COLUMN']),
                      crs='EPSG:4326' # GCS (WGS 84)
                      )
```

`gpd.points_from_xy` is equivalent to XY Table to Point in ArcGIS Pro. It creates a GeoSeries of geometries from x, y (and optionally z) coordinates.

In [ ]:
# Convert the dictionary into a GeoDataFrame with points as the geometry
origin_dest_gdf = gpd.GeoDataFrame(location_dict, geometry=gpd.points_from_xy(location_dict['Longitude'], location_dict['Latitude']), crs="EPSG:4326")
origin_dest_gdf

In [ ]:
# Explore funciton allow us to visualize the GeoDataFrame with an interactive map
origin_dest_gdf.explore()

## 1.3. Change coordinate system to Projected Coordinate System (PCS) to calculate distances in meters

In [ ]:
# Original CRS is WGS84
origin_dest_gdf.crs

In [ ]:
origin_dest_gdf_proj = origin_dest_gdf.to_crs(epsg=32614) # Project to UTM Zone 14N
origin_dest_gdf_proj

In [ ]:
origin_dest_gdf_proj.crs

In [ ]:
# geometry column in GeoDataFrame has a special attribute called x and y that can be used to extract the coordinates
# In this case, the unit of results is in meters as the GeoDataFrame is projected to UTM zone 14N
print(origin_dest_gdf_proj.at[0, 'geometry'].x, origin_dest_gdf_proj.at[0, 'geometry'].y)

In [ ]:
print(origin_dest_gdf_proj.at[1, 'geometry'].x, origin_dest_gdf_proj.at[1, 'geometry'].y)

 ## 1.4. Calculate Euclidean distance between two points
 
 Equation to calculate Euclidean distance
 
 $$
 d\left( p,q\right)   = \sqrt {\sum _{i=1}^{n}  \left( q_{i}-p_{i}\right)^2 } 
 $$

In [ ]:
# Euclidean distance
math.sqrt((origin_dest_gdf_proj.at[0, 'geometry'].x - origin_dest_gdf_proj.at[1, 'geometry'].x)**2 
          + (origin_dest_gdf_proj.at[0, 'geometry'].y - origin_dest_gdf_proj.at[1, 'geometry'].y)**2
          )

In [ ]:
# You can simply use distance method to calculate the distance between two points
origin_dest_gdf_proj.at[0, 'geometry'].distance(origin_dest_gdf_proj.at[1, 'geometry'])

---
### *Exercise*
1. (3 points) Find the longitude and latitude of the following two addresses using geopy package. 

**Origin Address (O'Kelly at UND)**: 221 Centennial Drive, Grand Forks, ND 58202 <br>
**Destination Address (Target)**: 3601 32nd Avenue South, Grand Forks, ND, 58201

```python
import geopy # Package to geocode addresses

# Initialize the geocoding engine
geolocator = geopy.geocoders.Nominatim(user_agent="my_first_geocoder")

# Geocode two addresses
origin = geolocator.geocode(`Origin Address`)
destination = geolocator.geocode(`Destination Address`)

print(f'|Origin| Longitude: {origin.longitude}, Latitude: {origin.latitude}')
print(f'|Destination| Longitude: {destination.longitude}, Latitude: {destination.latitude}')
```
---

In [ ]:
# # Your code here
# Initialize the geocoding engine
geolocator = geopy.geocoders.Nominatim(user_agent="my_first_geocoder")

# Geocode two addresses
origin = geolocator.geocode(`Origin Address`)
destination = geolocator.geocode(`Destination Address`)

print(f'|Origin| Longitude: {origin.longitude}, Latitude: {origin.latitude}')
print(f'|Destination| Longitude: {destination.longitude}, Latitude: {destination.latitude}')

In [ ]:
""" Test code for the previous function. 
This cell should NOT give any errors when it is run."""

assert round(origin.longitude, 4) == -97.0699
assert round(origin.latitude, 4) == 47.9208
assert round(destination.longitude, 4) == -97.0801
assert round(destination.latitude, 4) == 47.8865

print("Success!")

---
### *Exercise*
2. (5 points) Calculate Euclidean Distance between two points from GeoDataFrame. 
* Create GeoDataFrame from a dictionary that stores the geocoded results of the two addresses and save it into `geocoded_gdf`.
* Project GeoDataFrame(`geocoded_gdf`) to UTM Zone 14N (epsg code: 32614).
* Calculate Euclidean Distance using the GeoDataFrame.distance method and save the distance result into `euclidean_dist_two_points`.

```python
# Create a dictionary to store the geocoded results
geocoded_dict = {
    'Name': ['Origin', 'Destination'],
    'Latitude': [origin.latitude, destination.latitude],  # Enter variables here
    'Longitude': [origin.longitude, destination.longitude]  # Enter variables here
}

# Create a GeoDataFrame to store the geocoded results
geocoded_gdf = gpd.GeoDataFrame(geocoded_dict, 
                                geometry=gpd.points_from_xy(`LONGITUDE COLUMN OF A GEODATAFRAME`,
                                                            `LATITUDE COLUMN OF A GEODATAFRAME`), 
                                crs="EPSG:4326" # WGS 84
                                )

# Project the GeoDataFrame to UTM Zone 14N
geocoded_gdf = geocoded_gdf.to_crs(epsg=`EPSG CODE`) # Project to UTM Zone 14N (epsg code: 32614)

# Calculate the Euclidean distance between two points using the GeoDataFrame.distance method
euclidean_dist_two_points = `ORIGIN POINT`.distance(`DESTINATION POINT`)

print(euclidean_dist_two_points)
```
---



In [ ]:
# # Your code here

# Create a dictionary to store the geocoded results
geocoded_dict = {
    'Name': ['Origin', 'Destination'],
    'Latitude': [origin.latitude, destination.latitude],  # Enter variables here
    'Longitude': [origin.longitude, destination.longitude]  # Enter variables here
}

# Create a GeoDataFrame to store the geocoded results
geocoded_gdf = gpd.GeoDataFrame(geocoded_dict, 
                                geometry=gpd.points_from_xy(`LONGITUDE COLUMN OF A GEODATAFRAME`,
                                                            `LATITUDE COLUMN OF A GEODATAFRAME`), 
                                crs="EPSG:4326" # WGS 84
                                )

# Project the GeoDataFrame to UTM Zone 14N
geocoded_gdf = geocoded_gdf.to_crs(epsg=`EPSG CODE`) # Project to UTM Zone 14N (epsg code: 32614)

# Calculate the Euclidean distance between two points using the GeoDataFrame.distance method
euclidean_dist_two_points = `ORIGIN POINT`.distance(`DESTINATION POINT`)

print(euclidean_dist_two_points)

In [ ]:
""" Test code for the previous function. 
This cell should NOT give any errors when it is run."""

assert geocoded_gdf.shape == (2, 4)
assert geocoded_gdf.crs == "EPSG:32614"
assert round(euclidean_dist_two_points) == 3882

print("Success!")

# 2. Calculate accessible census blocks using Euclidean distance

## 2.1. Load census block data and geocode the stadium address

In [ ]:
# Initate a geocoding engine
geolocator = geopy.geocoders.Nominatim(user_agent="my_first_geocoder")

# Geocode the address of Ralph Engelstad Arena
stadium = geolocator.geocode("775 Hamline St, Grand Forks, ND 58203")

# Geocoded result is stored in the latitude and longitude attributes
stadium_gdf = gpd.GeoDataFrame({'Name': ['Ralph Engelstad Arena'], 
                                'Latitude': [stadium.latitude], 
                                'Longitude': [stadium.longitude]}, 
                                geometry=gpd.points_from_xy([stadium.longitude], [stadium.latitude]), 
                                crs="EPSG:4326"
                                )
stadium_gdf

In [ ]:
# Load the census block data
gf_block = gpd.read_file('./data/gf_census_block.geojson')
gf_block.head(3)

In [ ]:
# Plot the census block and the stadium
fig, ax = plt.subplots(figsize=(7,7))
gf_block.plot(color='lightgrey', edgecolor='grey', ax=ax)
stadium_gdf.plot(ax=ax, color='green')
plt.show()

## 2.2. Change coordinate system to Projected Coordinate System (PCS) to calculate distances in meters

In [ ]:
# `crs` attribute of the GeoDataFrame has coordinate reference system information
gf_block.crs

In [ ]:
stadium_gdf.crs

In [ ]:
# Again, we use to_crs method to project the GeoDataFrame to UTM Zone 14N
# This coordinate system has the unit in meters

stadium_gdf = stadium_gdf.to_crs(epsg=32614) # Project to UTM Zone 14N
gf_block = gf_block.to_crs(epsg=32614) # Project to UTM Zone 14N

In [ ]:
# Check the crs attribute again..
gf_block.crs

In [ ]:
stadium_gdf.crs

## 2.3. Create a buffer around the stadium to calculate Euclidean distance

Syntax to create a buffer around a point
```python
gdf = gdf.buffer(`RADIUS IN METERS`) 

OR

gdf = gdf['geometry'].buffer(`RADIUS IN METERS`)
```

In [ ]:
stadium_2km_buffer = stadium_gdf.copy() # Copy the stadium GeoDataFrame (Optional Step)

# Create a 2km buffer around the stadium
stadium_2km_buffer['geometry'] = stadium_2km_buffer['geometry'].buffer(2000)
stadium_2km_buffer


In [ ]:
# Plot the census block, the stadium, and the 2km buffer
fig, ax = plt.subplots(figsize=(7,7))
gf_block.plot(color='lightgrey', edgecolor='grey', ax=ax)
stadium_gdf.plot(ax=ax, color='green')
stadium_2km_buffer.boundary.plot(ax=ax, color='green', alpha=0.5)
plt.show()

## 2.4. Select census blocks that are within the buffer

Spatial operations methods in GeoPandas: <br>
- `intersects` method: Selects all rows that intersect a single geometry. <br>
- `within` method: Selects all rows that are within a single geometry. <br>
- `disjoint` method: Selects all rows that are disjoint to a single geometry. <br>

Syntax:
```python
`GEOMETRY COLUMN`.intersects(`A SINGLE GEOMETRY`) # intersect operation (can be replaced with within or disjoint)
```

Spatial operations in GeoPandas can be performed using the `loc` method. The loc method syntax is `df.loc[row condition, column condition]`. The following codes is the completed code to select a given geodataframe based on a spatial operation.<br>

Syntax:
```python
`SELECTED_GDF` = `GEODATAFRAME`.loc[`GEODATAFRAME_A`['geometry'].intersects(`A SINGLE GEOMETRY`)] # Intersects operation
```

Kinds of spatial operations: <br>
![](https://www.researchgate.net/profile/Tran-Dang-2/publication/286609202/figure/fig2/AS:925090808086531@1597570244506/Eight-OGC-spatial-relations-used-in-GeoXACML_W640.jpg)

In [ ]:
# Select a geometry only
stadium_2km_buffer['geometry'][0]

In [ ]:
# Query whether gf_block geometries intersect with the stadium_2km_buffer
gf_block.intersects(stadium_2km_buffer['geometry'][0])

In [ ]:
# Select the census blocks that intersect with the 2km buffer
block_2km_access = gf_block.loc[gf_block.intersects(stadium_2km_buffer['geometry'][0])]
block_2km_access

In [ ]:
# Select the census blocks that do not intersect (i.e., disjoint) with the 2km buffer 
block_2km_no_access = gf_block.loc[gf_block.disjoint(stadium_2km_buffer['geometry'][0])]
block_2km_no_access

In [ ]:
# You can also use the `~` (tilde) operator with intersects operation to select the disjoint blocks
gf_block.loc[~gf_block.intersects(stadium_2km_buffer['geometry'][0])]

In [ ]:
# Plot results

# Within operation results
block_2km_access_2 = gf_block.loc[gf_block.within(stadium_2km_buffer['geometry'][0])]
block_2km_no_access_2 = gf_block.loc[~gf_block.within(stadium_2km_buffer['geometry'][0])]

# Intersects result
fig, axes = plt.subplots(1, 2, figsize=(15, 7))
block_2km_access.plot(color='lightgreen', ax=axes[0])
block_2km_no_access.plot(color='lightcoral', ax=axes[0])
stadium_gdf.plot(ax=axes[0], color='green')
stadium_2km_buffer.boundary.plot(ax=axes[0], color='green', alpha=0.5)
axes[0].set_title(f'Intersects Operation ({block_2km_access.shape[0]} Blocks Selected)')

# Within result
block_2km_access_2.plot(color='lightgreen', ax=axes[1])
block_2km_no_access_2.plot(color='lightcoral', ax=axes[1])
stadium_gdf.plot(ax=axes[1], color='green')
stadium_2km_buffer.boundary.plot(ax=axes[1], color='green', alpha=0.5)
axes[1].set_title(f'Within Operation ({block_2km_access_2.shape[0]} Blocks Selected)')

plt.show()

---
### *Exercise*
3. (5 points) Create two GeoDataFrame (block_1km_access, block_1km_no_access) to find out the census blocks that are within and outside of the 1km buffer around the stadium. 

    output: 
    - block_1km_access: GeoDataFrame of census blocks that falls within the 1km buffer from the stadium.
    - block_1km_no_access: GeoDataFrame of census blocks that falls outside of the 1km buffer from the stadium.

    Hint: Use the `within` method to select the blocks that falls within the 1km buffer.
   
   Summary of codes:
```python
# Make a copy from the original stadium GeoDataFrame
stadium_buffer = stadium_gdf.copy()

# Create a buffer around the stadium
stadium_buffer['geometry'] = stadium_gdf['geometry'].buffer(`RADIUS IN METERS`) # create buffer

# Select census blocks that are within the buffer
block_1km_access = `CENSUS BLOCK GDF`.loc[`CENSUS BLOCK GDF`['geometry'].`SPATIAL OPERATION`(`A SINGLE GEOMETRY`)] 
block_1km_no_access = `HINT: USE TILDE`
```

---

In [ ]:
# # Your code here

# Make a copy from the original stadium GeoDataFrame
stadium_buffer = stadium_gdf.copy()

# Create a buffer around the stadium
stadium_buffer['geometry'] = stadium_gdf['geometry'].buffer(`RADIUS IN METERS`) # create buffer

# Select census blocks that are within the buffer
block_1km_access = `CENSUS BLOCK GDF`.loc[`CENSUS BLOCK GDF`['geometry'].`SPATIAL OPERATION`(`A SINGLE GEOMETRY`)] 
block_1km_no_access = `HINT: USE TILDE`

In [ ]:
""" Test code for the previous function. 
This cell should NOT give any errors when it is run."""

assert round(stadium_1km_buffer['geometry'][0].area) == 3136548
assert block_1km_access.shape[0] == 54
assert block_1km_no_access.shape[0] == 1036

print("Success!")

# 3. Calculate Manhattan Distance

## 3.1. Download street network data from OpenStreetMap

Syntax to download street network data from OpenStreetMap
```python
network = ox.graph_from_place(`AN ADDRESS OF GEOCODEABLE ADMINISTRATIVE REGION`, network_type='drive') 
```

Source: https://osmnx.readthedocs.io/en/stable/user-reference.html#osmnx.graph.graph_from_place

In [ ]:
# Obtain OSM Network in the city of Grand Forks
G = ox.graph_from_place('Grand Forks, ND, USA', network_type='drive', simplify=True)

# Plot the network
ox.plot_graph(G)


In [ ]:
# In case the download is failed, you can load the graph from the local file

# G= ox.load_graphml('./data/grand_forks_road.graphml')
# ox.plot_graph(G)

In [ ]:
# The downloaded graph is a MultiDiGraph, and it can be converted into GeoDataFrames using ox.graph_to_gdfs function
nodes, edges = ox.graph_to_gdfs(G, nodes=True, edges=True)
nodes

OSM highway types: https://wiki.openstreetmap.org/wiki/Key:highway

In [ ]:
edges

## 3.2. Snap the stadium location to the nearest node on the street network

Network dataset consists of nodes and edges. Nodes are points where edges meet. Edges are lines that represent streets. <br>
![](https://upload.wikimedia.org/wikipedia/commons/2/2f/Small_Network.png)

To calculate Manhattan distance, we need to snap the stadium location to the nearest node on the street network using the `osmnx` package.
Syntax: 
```python
nearest_node = ox.distance.nearest_nodes(`NETWORK`, `POINT LONGITUDE`, `POINT LATITUDE`)
```

Source: https://osmnx.readthedocs.io/en/stable/user-reference.html#osmnx.distance.nearest_nodes

In [ ]:
# Initate a geocoding engine
geolocator = geopy.geocoders.Nominatim(user_agent="my_first_geocoder")

# Geocode the address of Ralph Engelstad Arena
stadium = geolocator.geocode("775 Hamline St, Grand Forks, ND 58203")
print('Stadium Location:', stadium.latitude, stadium.longitude)

# Geocoding a different location (a drom at University of North Dakota)
dorm = geolocator.geocode("3601 University Ave, Grand Forks, ND 58202")
print('Dorm Location: ', dorm.latitude, dorm.longitude)

In [ ]:
stadium_nearest_node = ox.distance.nearest_nodes(G, stadium.longitude, stadium.latitude) # OSMID nearest to stadium
print("Nearest Node's OSMID from Stadium:", stadium_nearest_node)

dorm_nearest_node = ox.distance.nearest_nodes(G, dorm.longitude, dorm.latitude) # OSMID nearest to dorm
print("Nearest Node's OSMID from Dorm:", dorm_nearest_node)

In [ ]:
# Plot the offset between the stadium and the nearest node

fig, ax = plt.subplots(figsize=(7,7))

# Stadium location
nodes.loc[nodes.index == stadium_nearest_node].plot(ax=ax, color='blue')  # Snapping point
ax.plot(stadium.longitude, stadium.latitude, c='black', marker='o')  # Original point

# Dorm location
nodes.loc[nodes.index == dorm_nearest_node].plot(ax=ax, color='red')  # Snapping point
ax.plot(dorm.longitude, dorm.latitude, c='black', marker='o')  # Original point

# The below is just for plotting purposes
temp_buffer = stadium_gdf.buffer(1500)
temp_buffer_gcs = temp_buffer.to_crs(epsg=4326) # Project to WGS84
edges.loc[edges.geometry.intersects(temp_buffer_gcs.geometry[0])].plot(ax=ax, color='grey', linewidth=0.5)

## 3.3. Calculate Manhattan distance between the stadium and the dorm 

In [ ]:
# Reminder of how to create a GeoDataFrame from a dictionary

# You can save the geocoded result in a dictionary to be converted into a (Geo)DataFrame
location_dict = {
    'Name': ['Dorm', 'Stadium'],
    'Latitude': [dorm.latitude, stadium.latitude],
    'Longitude': [dorm.longitude, stadium.longitude]
}

# Convert the dictionary into a GeoDataFrame with points as the geometry
origin_dest_gdf = gpd.GeoDataFrame(location_dict, geometry=gpd.points_from_xy(location_dict['Longitude'], location_dict['Latitude']), crs="EPSG:4326")

origin_dest_gdf_proj = origin_dest_gdf.to_crs(epsg=32614) # Project to UTM Zone 14N

# You can simply use distance method to calculate the distance between two points
euclidean_distance = origin_dest_gdf_proj.at[0, 'geometry'].distance(origin_dest_gdf_proj.at[1, 'geometry'])
print('Euclidean Distance between the stadium ann dorm:', euclidean_distance)

You can calculate the Manhattan distance between two points using the following code.

Syntax: 
```python
nx.shortest_path_length(`NETWORK`, `NEAREST NODE OF ORIGIN`, `NEAREST NODE OF DESTINATION`, weight='length')
```

Source: https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.shortest_paths.generic.shortest_path_length.html

In [ ]:
# Previous code to snap the stadium and dorm to the nearest node
stadium_nearest_node = ox.distance.nearest_nodes(G, stadium.longitude, stadium.latitude) # OSMID nearest to stadium
print("Nearest Node's OSMID from Stadium:", stadium_nearest_node)

dorm_nearest_node = ox.distance.nearest_nodes(G, dorm.longitude, dorm.latitude) # OSMID nearest to dorm
print("Nearest Node's OSMID from Dorm:", dorm_nearest_node)

In [ ]:
# Calculate the shortest path between the stadium and dorm
manhattan_distance = nx.shortest_path_length(G, 75962224, 75861583, weight='length')

# The code below would give you the same result as `dorm_nearest_node` and `stadium_nearest_node` already has the nearest node's OSMID
# manhattan_distance = nx.shortest_path_length(G, dorm_nearest_node, stadium_nearest_node, weight='length')

print('Manhattan Distance between the stadium and dorm:', manhattan_distance)

You can also find the route between two points using the following code.
Note: The routes is a list of nodes that are connected to each other.

Syntax: 
```python
routes = nx.shortest_path(`NETWORK`, `NEAREST NODE OF ORIGIN`, `NEAREST NODE OF DESTINATION`, weight='length')

```

Source: https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.shortest_paths.generic.shortest_path.html

In [ ]:
routes = nx.shortest_path(G, 75962224, 75861583, weight='length')
routes

In [ ]:
od_list = [[routes[i], routes[i+1]] for i in range(len(routes)-1)]
routes_gdf = gpd.GeoDataFrame(geometry=[edges.loc[(u, v)].geometry.iloc[0] for u, v in od_list], crs=edges.crs)
routes_gdf

In [ ]:
# Plot the offset between the stadium and the nearest node

fig, ax = plt.subplots(figsize=(7,7))

# Stadium location
nodes.loc[nodes.index == stadium_nearest_node].plot(ax=ax, color='blue')  # Snapping point
ax.plot(stadium.longitude, stadium.latitude, c='black', marker='o')  # Original point

# Dorm location
nodes.loc[nodes.index == dorm_nearest_node].plot(ax=ax, color='red')  # Snapping point
ax.plot(dorm.longitude, dorm.latitude, c='black', marker='o')  # Original point

# Euclidean distance
ax.plot([stadium.longitude, dorm.longitude], [stadium.latitude, dorm.latitude], color='black')

# Manhattan distance
routes_gdf.plot(ax=ax, color='red', linewidth=3)

# The below is just for plotting purposes
temp_buffer = stadium_gdf.buffer(1500)
temp_buffer_gcs = temp_buffer.to_crs(epsg=4326) # Project to WGS84
edges.loc[edges.geometry.intersects(temp_buffer_gcs.geometry[0])].plot(ax=ax, color='grey', linewidth=0.5)

---
### *Exercise*
4. (5 points) Find the nearest OSM ID from the following two addresses.

**Origin Address (O'Kelly at UND)**: 221 Centennial Drive, Grand Forks, ND 58202 <br>
**Destination Address (Target)**: 3601 32nd Avenue South, Grand Forks, ND, 58201

Investigate how to snap the origin and destination locations to their nearest node on the street network using the `osmnx` package.
```python
nearest_node = ox.distance.nearest_nodes(`NETWORK`, `POINT LONGITUDE`, `POINT LATITUDE`)

# This code allows you to find the longitude and latitude of the two addresses
# Initialize the geocoding engine
geolocator = geopy.geocoders.Nominatim(user_agent="my_first_geocoder")

# Geocode two addresses
origin = geolocator.geocode("221 Centennial Drive, Grand Forks, ND 58202")
destination = geolocator.geocode("3601 32nd Avenue South, Grand Forks, ND, 58201")

print(f'|Origin| Longitude: {origin.longitude}, Latitude: {origin.latitude}')
print(f'|Destination| Longitude: {destination.longitude}, Latitude: {destination.latitude}')

# Your code here
origin_nearest_node = ox.distance.nearest_nodes(`NETWORK`, `POINT LONGITUDE`, `POINT LATITUDE`)
destination_nearest_node = ox.distance.nearest_nodes(`NETWORK`, `POINT LONGITUDE`, `POINT LATITUDE`)

print("Origin's Nearest Node's OSMID:", origin_nearest_node)
print("Destination's Nearest Node's OSMID:", destination_nearest_node)
```
---

In [ ]:
# # This code allows you to find the longitude and latitude of the two addresses
# Initialize the geocoding engine
geolocator = geopy.geocoders.Nominatim(user_agent="my_first_geocoder")

# Geocode two addresses
origin = geolocator.geocode("221 Centennial Drive, Grand Forks, ND 58202")
destination = geolocator.geocode("3601 32nd Avenue South, Grand Forks, ND, 58201")

print(f'|Origin| Longitude: {origin.longitude}, Latitude: {origin.latitude}')
print(f'|Destination| Longitude: {destination.longitude}, Latitude: {destination.latitude}')

# Your code here
origin_nearest_node = ox.distance.nearest_nodes(`NETWORK`, `POINT LONGITUDE`, `POINT LATITUDE`)
destination_nearest_node = ox.distance.nearest_nodes(`NETWORK`, `POINT LONGITUDE`, `POINT LATITUDE`)

print("Origin's Nearest Node's OSMID:", origin_nearest_node)
print("Destination's Nearest Node's OSMID:", destination_nearest_node)

In [ ]:
""" Test code for the previous function. 
This cell should NOT give any errors when it is run."""

assert origin_nearest_node == 75958974
assert destination_nearest_node == 75860718

print("Success!")

---
### *Exercise*
5. (2 points) Calculate the Manhattan Distance between the Origin and Destination addresses and save the result to `manhattan_dist_two_points`.

Syntax to calculate the Manhattan distance between two points
```python
manhattan_dist_two_points = nx.shortest_path_length(`NETWORK`, `NEAREST NODE OF ORIGIN`, `NEAREST NODE OF DESTINATION`, weight='length')
print(manhattan_distance)
```
---

In [ ]:
# # Your code here
manhattan_dist_two_points = nx.shortest_path_length(`NETWORK`, `NEAREST NODE OF ORIGIN`, `NEAREST NODE OF DESTINATION`, weight='length')
print(manhattan_distance)

In [ ]:
""" Test code for the previous function. 
This cell should NOT give any errors when it is run."""

assert round(manhattan_distance) == 4748

print("Success!")

In [ ]:
# Plot the difference between Manhattan and Euclidean distances of the two points
fig, ax = plt.subplots(figsize=(7,7))

ax.plot(origin.longitude, origin.latitude, c='black', marker='o')  # Origin point
ax.plot(destination.longitude, destination.latitude, c='black', marker='o')  # Destination point

nodes.loc[nodes.index == origin_nearest_node].plot(ax=ax, color='red')  # Snapping point
nodes.loc[nodes.index == destination_nearest_node].plot(ax=ax, color='blue')  # Snapping point

ax.plot([origin.longitude, destination.longitude], [origin.latitude, destination.latitude], color='black')  # Euclidean distance

_routes = nx.shortest_path(G, origin_nearest_node, destination_nearest_node, weight='length')
_od_list = [[_routes[i], _routes[i+1]] for i in range(len(_routes)-1)]
_routes_gdf = gpd.GeoDataFrame(geometry=[edges.loc[(u, v)].geometry.iloc[0] for u, v in _od_list], crs=edges.crs)
_routes_gdf.plot(ax=ax, color='red', linewidth=3)  # Manhattan distance

edges.plot(ax=ax, color='grey', linewidth=0.5)

# 4. Calculate accessible census blocks using Manhattan Distance

In [ ]:
# Reload the stadium location 

# Initate a geocoding engine
geolocator = geopy.geocoders.Nominatim(user_agent="my_first_geocoder")

# Geocode the address of Ralph Engelstad Arena
stadium = geolocator.geocode("775 Hamline St, Grand Forks, ND 58203")

# Previous code to snap the stadium and dorm to the nearest node
stadium_nearest_node = ox.distance.nearest_nodes(G, stadium.longitude, stadium.latitude) # OSMID nearest to stadium
print("Nearest Node's OSMID from Stadium:", stadium_nearest_node)

## 4.1. Find the accessible nodes within a given distance

Syntax to find the accessible nodes within a given distance
```python
`ACCESSIBLE_NODE_DICTIONARY` = nx.single_source_dijkstra_path_length(`NETWORK`, `NEAREST NODE OF ORIGIN`, `DISTANCE IN METERS`, weight='length')
```

In [ ]:
temp_manhattan_dist = nx.single_source_dijkstra_path_length(G, stadium_nearest_node, cutoff=2000, weight='length')
temp_manhattan_dist

In [ ]:
# Get the keys of the dictionary from temp_manhattan_dist
temp_manhattan_dist.keys()

In [ ]:
# Check whether the nodes are in the keys of the dictionary
nodes.index.isin(list(temp_manhattan_dist.keys()))

In [ ]:
# Select the nodes that are within 2km from the stadium
access_node_gdf = nodes.loc[nodes.index.isin(list(temp_manhattan_dist.keys()))]
access_node_gdf

In [ ]:
fig, ax = plt.subplots(figsize=(7,7))

ax.plot(stadium.longitude, stadium.latitude, c='green', marker='o')  # Stadium point
access_node_gdf.plot(ax=ax, color='black', markersize=5)  # Nodes within 2km from the stadium

# The below is just for plotting purposes
_temp_buffer = stadium_gdf.buffer(2000)
_temp_buffer_gcs = _temp_buffer.to_crs(epsg=4326) # Project to WGS84
edges.loc[edges.intersects(_temp_buffer_gcs[0])].plot(ax=ax, color='grey', linewidth=0.5)  # Roads within 2km from the stadium

## 4.2. Create Convex Hull from the accessible nodes

The convex hull of a set of points is the smallest convex set that contains all the points. 


![](https://miro.medium.com/v2/resize:fit:677/1*F4IUmOJbbLMJiTgHxpoc7Q.png)


In [ ]:
# GeoDataFrame of accessible nodes
access_node_gdf

In [ ]:
# Dissove the GeoDataFrame to create a single geometry
access_node_gdf.unary_union

In [ ]:
# In the form of well-known text (WKT)
access_node_gdf.unary_union.wkt

In [ ]:
# How to create a convex hull from the GeoDataFrame
access_node_gdf.unary_union.convex_hull

In [ ]:
# Convex hull needs to be converted into a GeoDataFrame to be plotted. 
convex_hull_gdf = gpd.GeoDataFrame({'geometry': [access_node_gdf.unary_union.convex_hull]}, crs=access_node_gdf.crs)
convex_hull_gdf

In [ ]:
access_node_gdf.crs

In [ ]:
fig, ax = plt.subplots(figsize=(7,7))

ax.plot(stadium.longitude, stadium.latitude, c='green', marker='o')  # Stadium point
access_node_gdf.plot(ax=ax, color='black', markersize=5)  # Nodes within 2km from the stadium
convex_hull_gdf.boundary.plot(ax=ax, color='green')  # Convex hull

# The below is just for plotting purposes
_temp_buffer = stadium_gdf.buffer(2000)
_temp_buffer_gcs = _temp_buffer.to_crs(epsg=4326) # Project to WGS84
edges.loc[edges.intersects(_temp_buffer_gcs[0])].plot(ax=ax, color='grey', linewidth=0.5)  # Roads within 2km from the stadium

## 4.3. Select census blocks that are within the convex hull

Syntax to select census blocks that are within the convex hull
```python
`RETURN GDF` = `CENSUS BLOCK GDF`.loc[`CENSUS BLOCK GDF`['geometry'].within(`A SINGLE GEOMETRY`)] 
```

In [ ]:
# Reload the census blocks
gf_block_gcs = gpd.read_file('./data/gf_census_block.geojson')
gf_block_gcs.head(3)

In [ ]:
# Select the census blocks that are within the convex hull
block_within_convex_hull = gf_block_gcs.loc[gf_block_gcs.within(convex_hull_gdf.geometry[0])]

# Select the census blocks that are outside the convex hull using the ~ (tilde) operator
block_outside_convex_hull = gf_block_gcs.loc[~gf_block_gcs.within(convex_hull_gdf.geometry[0])]

In [ ]:
# Plot results

fig, axes = plt.subplots(1, 2, figsize=(15, 7))

# Euclidean distance results
block_2km_access_2.plot(color='lightgreen', ax=axes[0])
block_2km_no_access_2.plot(color='lightcoral', ax=axes[0])
stadium_gdf.plot(ax=axes[0], color='green')
stadium_2km_buffer.boundary.plot(ax=axes[0], color='green', alpha=0.5)
axes[0].set_title(f'Within 2KM Euclidean Distance ({block_2km_access_2.shape[0]} Blocks Selected)')

axes[1].plot(stadium.longitude, stadium.latitude, c='green', marker='o')  # Stadium point
convex_hull_gdf.boundary.plot(ax=axes[1], color='green')  # Convex hull
block_within_convex_hull.plot(ax=axes[1], color='lightgreen')  # Blocks within the convex hull
block_outside_convex_hull.plot(ax=axes[1], color='lightcoral')  # Blocks outside the convex hull
axes[1].set_title(f'Within 2KM Manhattan Distance ({block_within_convex_hull.shape[0]} Blocks Selected)')

plt.show()

In [ ]:
# Summary

# Initate a geocoding engine
geolocator = geopy.geocoders.Nominatim(user_agent="my_first_geocoder")

# Geocode the address of Ralph Engelstad Arena
stadium = geolocator.geocode("775 Hamline St, Grand Forks, ND 58203")

# Snap the stadium and dorm to the nearest OSM node
stadium_nearest_node = ox.distance.nearest_nodes(G, stadium.longitude, stadium.latitude) # OSMID nearest to stadium
print("Nearest Node's OSMID from Stadium:", stadium_nearest_node)

# Select the nodes that are within 2km from the stadium
temp_manhattan_dist = nx.single_source_dijkstra_path_length(G, stadium_nearest_node, cutoff=2000, weight='length')

# Select the node geometry that are within 2km from the stadium
access_node_gdf = nodes.loc[nodes.index.isin(list(temp_manhattan_dist.keys()))]

# Convex hull needs to be converted into a GeoDataFrame to be plotted. 
convex_hull_gdf = gpd.GeoDataFrame({'geometry': [access_node_gdf.unary_union.convex_hull]}, crs=access_node_gdf.crs)

# Select census block that are within / outside the convex hull
block_within_convex_hull = gf_block_gcs.loc[gf_block_gcs.within(convex_hull_gdf.geometry[0])]
block_outside_convex_hull = gf_block_gcs.loc[~gf_block_gcs.within(convex_hull_gdf.geometry[0])]

---
### *Exercise*
6. (5 points) Calculate accessible locations via Manhattan Distance of 1km.
* Get the nearest OSM Node ID from the stadium longitude (-97.06912354887743) and latitude (47.927281818943364) and save the result to `stadium_osmid`.
* Calculate the accessible OSM nodes within 1km from the stadium and save the result to `nodes_1km`
* Get the geometry of accessible nodes from `nodes` GeoDataFrame and save the result to `nodes_1km_gdf`

```python
# Get the nearest OSM Node ID from the stadium longitude and latitude
stadium_osmid = ox.distance.nearest_nodes(G, `LONGITUDE`, `LATITUDE`)

# Calculate the accessible OSM nodes within 1km from the stadium
nodes_1km = nx.single_source_dijkstra_path_length(G, `OSM ID OF THE STADIUM`, cutoff=`DISTANCE IN METERS`, weight='length')

# Get the geometry of accessible nodes from `nodes` GeoDataFrame
nodes_1km_gdf = nodes.loc[]
```

---

In [ ]:
# # Your code here

# Get the nearest OSM Node ID from the stadium longitude and latitude
stadium_osmid = ox.distance.nearest_nodes(G, `LONGITUDE`, `LATITUDE`)

# Calculate the accessible OSM nodes within 1km from the stadium
nodes_1km = nx.single_source_dijkstra_path_length(G, `OSM ID OF THE STADIUM`, cutoff=`DISTANCE IN METERS`, weight='length')

# Get the geometry of accessible nodes from `nodes` GeoDataFrame
nodes_1km_gdf = nodes.loc[]

In [ ]:
""" Test code for the previous function. 
This cell should NOT give any errors when it is run."""

assert stadium_osmid == 75861583
assert len(nodes_1km) == 80
assert round(nodes_1km[75921119]) == 995
assert nodes_1km_gdf.shape[0] == 80

print("Success!")

---
### *Exercise*

7. (5 points) Create a Convex Hull and select accessible census blocks
* Create a convex hull from `nodes_1km_gdf` and save the result to `ch_1km`
* Select census blocks (gf_block_gcs) that are WITHIN the convex hull (ch_1km) and save the result to `block_access_1km`
* Select census blocks (gf_block_gcs) that are OUTSIDE the convex hull (ch_1km) and save the result to `block_no_access_1km`

```python
# Get Convex Hull from the accessible node geometries
ch_1km = nodes_1km_gdf.`CREATE MULTIPOINT GEOMETRY`.`METHOD TO CREATE CONVEX HULL`

# Select the census blocks that are within the convex hull
block_access_1km = `CENSUS BLOCK GDF`.loc[`CENSUS BLOCK GDF`['geometry'].`SPATIAL OPERATION`(`A SINGLE GEOMETRY`)] 

# Select the census blocks that are outside the convex hull (take advantage of the `~` operator)
block_no_access_1km = `CENSUS BLOCK GDF`.loc[`CENSUS BLOCK GDF`['geometry'].`SPATIAL OPERATION`(`A SINGLE GEOMETRY`)] 

```
---

In [ ]:
# # Your code here

# Get Convex Hull from the accessible node geometries
ch_1km = nodes_1km_gdf.`CREATE MULTIPOINT GEOMETRY`.`METHOD TO CREATE CONVEX HULL`

# Select the census blocks that are within the convex hull
block_access_1km = `CENSUS BLOCK GDF`.loc[`CENSUS BLOCK GDF`['geometry'].`SPATIAL OPERATION`(`A SINGLE GEOMETRY`)] 

# Select the census blocks that are outside the convex hull (take advantage of the `~` operator)
block_no_access_1km = `CENSUS BLOCK GDF`.loc[`CENSUS BLOCK GDF`['geometry'].`SPATIAL OPERATION`(`A SINGLE GEOMETRY`)] 


In [ ]:
""" Test code for the previous function. 
This cell should NOT give any errors when it is run."""

assert round(ch_1km.area, 6) == 0.000193
assert block_access_1km.shape[0] == 34
assert block_no_access_1km.shape[0] == 1056

print("Success!")

# 5. Calculate accessible area with multiple infrastructure

In [ ]:
# Grocery stores in Grand Forks
grocery_df = pd.read_csv('./data/grocery_gf.csv')
grocery_df

In [ ]:
# Calculate Geometry from the Longitude and Latitude
grocery_gdf = gpd.GeoDataFrame(grocery_df, geometry=gpd.points_from_xy(grocery_df.Longitude, grocery_df.Latitude), crs='EPSG:4326')
grocery_gdf.head(3)

In [ ]:
# Load census blocks
gf_block_gcs = gpd.read_file('./data/gf_census_block.geojson')
gf_block_gcs

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
gf_block_gcs.plot(ax=ax, color='lightgrey', edgecolor='grey')
grocery_gdf.plot(ax=ax, color='blue')

In [ ]:
# This function helps you to find the nearest OSM node from a given GeoDataFrame
# If geom type is point, it will take it without modification, but 
# IF geom type is polygon or multipolygon, it will take its centroid to calculate the nearest element. 

def find_nearest_osm(network, gdf):

    for idx, row in gdf.iterrows():
        if row.geometry.geom_type == 'Point':
            nearest_osm = ox.distance.nearest_nodes(network, 
                                                    X=row.geometry.x, 
                                                    Y=row.geometry.y
                                                   )
        elif row.geometry.geom_type == 'Polygon' or row.geometry.geom_type == 'MultiPolygon':
            nearest_osm = ox.distance.nearest_nodes(network, 
                                        X=row.geometry.centroid.x, 
                                        Y=row.geometry.centroid.y
                                       )
        else:
            print(row.geometry.geom_type)
            continue

        gdf.at[idx, 'nearest_osm'] = nearest_osm

    gdf['nearest_osm'] = gdf['nearest_osm'].astype(int)

    return gdf

In [ ]:
grocery_gdf = find_nearest_osm(G, grocery_gdf)
grocery_gdf

In [ ]:
convex_hull_gdf = gpd.GeoDataFrame({'geometry': []}, crs=grocery_gdf.crs)

for idx, row in grocery_gdf.iterrows():
    temp_nodes = nx.single_source_dijkstra_path_length(G, row['nearest_osm'], cutoff=1000, weight='length')
    temp_access_nodes = nodes.loc[nodes.index.isin(temp_nodes.keys()), 'geometry']
    convex_hull_gdf.at[idx, 'geometry'] = temp_access_nodes.unary_union.convex_hull
    convex_hull_gdf.at[idx, 'store_name'] = row['Name']

convex_hull_gdf.explore()

In [ ]:
grocery_access_1km = gf_block_gcs.loc[gf_block_gcs['geometry'].within(convex_hull_gdf.unary_union)]
grocery_no_access_1km = gf_block_gcs.loc[~gf_block_gcs['geometry'].within(convex_hull_gdf.unary_union)]

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7))

gpd.GeoDataFrame({'geometry': [convex_hull_gdf.unary_union]}, crs=grocery_gdf.crs).boundary.plot(ax=ax, color='blue', zorder=2)
edges.plot(ax=ax, color='black', linewidth=0.5, zorder=1)
grocery_gdf.plot(ax=ax, color='blue', markersize=10, zorder=3)
grocery_access_1km.plot(ax=ax, color='lightgreen', zorder=0)
grocery_no_access_1km.plot(ax=ax, color='lightcoral', zorder=0)

# Done